# Demonstration of shifty.py's "refcat" module
 - 20191106
 - Payne 
 
The refcat module facilitates use of Tonry's refcat2 catalog
    
    - https://archive.stsci.edu/prepds/atlas-refcat2/
    - http://adsabs.harvard.edu/cgi-bin/bib_query?arXiv:1809.09157

There is a "RefCat" class
 - This handles both datadownloads and calls to the catalog

This notebook:
 - Demonstrates the key components / classes / functions available in the refcat module
 - Acts as a complement to the tests of the same module. 

#### Import statements 

In [4]:
# -------------------------------------------------------------------------------------
# Third party imports
# -------------------------------------------------------------------------------------
import numpy as np 
import matplotlib.pyplot as plt
import os, sys
import glob 

import astropy
from astropy.io import fits
from astropy.time import Time
from astropy.units import allclose as quantity_allclose
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates.builtin_frames import FK5, ICRS, GCRS, GeocentricMeanEcliptic, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, GeocentricTrueEcliptic, BarycentricTrueEcliptic, HeliocentricTrueEcliptic, HeliocentricEclipticIAU76
from astropy.constants import R_sun, R_earth

# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------
from refcat import RefCat


### Instantiate RefCat object

In [7]:
# creation of RefCat object
RC = RefCat()

# examination of instantiated variables
# - these are defining some local data storage 
for k,v in RC.__dict__.items():
    print(k,v)


downloadable_files {'code': 'https://archive.stsci.edu/prepds/atlas-refcat2/refcat.c', 'man': 'https://archive.stsci.edu/prepds/atlas-refcat2/refcat.man', '20': 'https://archive.stsci.edu/hlsps/atlas-refcat2/orig/hlsp_atlas-refcat2_atlas_ccd_19-m-20_multi_v1_cat.tbz', '18': 'https://archive.stsci.edu/hlsps/atlas-refcat2/orig/hlsp_atlas-refcat2_atlas_ccd_17-m-18_multi_v1_cat.tbz', '17': 'https://archive.stsci.edu/hlsps/atlas-refcat2/orig/hlsp_atlas-refcat2_atlas_ccd_16-m-17_multi_v1_cat.tbz', '16': 'https://archive.stsci.edu/hlsps/atlas-refcat2/orig/hlsp_atlas-refcat2_atlas_ccd_00-m-16_multi_v1_cat.tbz', '19': 'https://archive.stsci.edu/hlsps/atlas-refcat2/orig/hlsp_atlas-refcat2_atlas_ccd_18-m-19_multi_v1_cat.tbz'}
refcat_codename refcat
refcat_filepath /Users/matthewjohnpayne/.shifty_data/refcat/refcat
refcat_dir /Users/matthewjohnpayne/.shifty_data/refcat
local_dir /Users/matthewjohnpayne/.shifty_data


### Use RefCat to download refcat catalog(s) 
 - Note that this takes a *long* time 
 - It is downloading a huge file 
 - Then it un-tars that huge file into a directory containing 10's of thousands of files 

The downloaded file(s) will be put into a directory defined by the variable ...
 - RC.refcat_dir

If the download-file already exists on disk then it will *NOT* be downloaded again

In [9]:
# Tonry's catalog can be downloaded in 5 chunks 
# Here we are downloading the 'brighter-than'16th-magnitude' chunk
# - This is implicitly using the keys in the RC.downloadable_files() dictionary to specify what you want to download
RC.download_refcat(16)

In [ ]:
# Here we are downloading all of the available catalogs 
RC.download_refcat( *list(RC.downloadable_files.keys()) )

### Use RefCat to download refcat code
 - This is quick 

In [ ]:
RC.download_refcode()

### Use RefCat to compile refcat code
 - This is quick (if it works) 
 - Requires that your machine have relevant (cc) compiler
 - I have *NOT* attempted to generalize beyond the method Tonry suggests 

In [ ]:
RC.compile_refcat()

### Use RefCat to search source catalogs 

In [13]:
 # The following is taken from refcat.man
# - Report all stars within a radius of 1 deg from RA, Dec 180,10.
# - Request input file extension ".rc2" explicitly and a header line
# - (1064 stars, output in ATLAS format):
# >>> refcat 180 10 -rad 1 -dir 00_m_16 -exten rc2 -hdr
# -----------------------------------------
ra = 180
dec = 10
rad = 1
mlim = 17
code = RC.refcat_filepath
dir = os.path.join(RC.refcat_dir, '00_m_16')
result = RC._read_refcat(ra, dec, code, dir,
                     rad=rad , mlim=mlim )

# check the results are as expected 
expectedkeys  = ['ra', 'dec', 'g', 'r', 'i', 'z', 'J', 'c', 'o']
if isinstance(result, dict) and np.all([k in result for k in expectedkeys]) : 
    print('All expected keys are in result-dictionary')

expected_number_sources = 1064
if np.all( [ k in expectedkeys and len(v) == expected_number_sources for k,v in result.items() ]):
    print('The expected number of source was found [%r]' % expected_number_sources)
    
# illustrate a sample of the results 
n_rows_to_print = 5
for key in expectedkeys:
    print('%10s' % key, end=',')
print()
for i in range(n_rows_to_print):
    for key in expectedkeys:
        print('%10f' % result[key][i], end=',')
    print()

All expected keys are in result-dictionary
The expected number of source was found [1064]
        ra,       dec,         g,         r,         i,         z,         J,         c,         o,
179.012931,  9.887627, 14.803000, 14.237000, 14.063000, 13.999000, 13.023000, 14.523000, 14.157000,
179.026188,  9.842122, 13.400000, 12.972000, 12.842000, 12.804000, 11.876000, 13.191000, 12.913000,
179.035762,  9.935784, 15.879000, 15.317000, 15.096000, 15.015000, 14.033000, 15.601000, 15.215000,
179.044683,  9.938728, 16.628000, 15.457000, 14.531000, 14.102000, 12.707000, 16.015000, 14.970000,
179.057159,  9.708921, 15.200000, 14.857000, 14.752000, 14.725000, 13.852000, 15.034000, 14.809000,
